# Predicting House Sale Prices

In this project we will work with housing data for the city of Ames, Iowa, United States from 2006 to 2010.

Let's start by setting up a pipeline of functions that will let us quickly iterate on different models

In [257]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [258]:
df = pd.read_csv('AmesHousing.tsv', delimiter="\t")

In [259]:
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [263]:
def transform_features(df):
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):  
    train = df[:1460]
    test = df[1460:]
    

    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop("SalePrice")
    model = LinearRegression()
    model.fit(train[features], train["SalePrice"])
    predictions = model.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

transform_df = transform_features(df)
final_df = select_features(transform_df)
rmse = train_and_test(final_df)

rmse

57088.25161263909

### Feature transformation

We'll remove numerical columns that contain more than 5% missing values.

In [265]:
missing_vals = df.isnull().sum()

In [266]:
missing_cols = missing_vals[missing_vals>len(df)*0.05].index
df = df.drop(missing_cols, axis=1)

For categorical columns we will remove all columns that have at least one mising value.

In [267]:
text_missing_vals = df.select_dtypes(include=['object']).isnull().sum()
text_missing_cols = text_missing_vals[text_missing_vals>0].index
df = df.drop(text_missing_cols, axis=1)

In [268]:
df.shape

(2930, 64)

For the remaining numerical columns we well fill in the `NaN` values with the most common value in that column since the mean for columns such as `Garage Cars` wouldn't make much sense.

In [269]:
num_cols = df.select_dtypes(include=['int', "float"]).isnull().sum().sort_values(ascending=False)

In [270]:
num_cols = num_cols[num_cols >= 1]

In [271]:
mode = df[num_cols.index].mode().to_dict('records')
mode

[{'Bsmt Full Bath': 0.0,
  'Bsmt Half Bath': 0.0,
  'Bsmt Unf SF': 0.0,
  'BsmtFin SF 1': 0.0,
  'BsmtFin SF 2': 0.0,
  'Garage Area': 0.0,
  'Garage Cars': 2.0,
  'Mas Vnr Area': 0.0,
  'Total Bsmt SF': 0.0}]

In [272]:
df = df.fillna(mode[0])

In [273]:
df.isnull().sum().value_counts(dropna=False)

0    64
dtype: int64

We'll calculate the span for the years the houses were built and remodeled.

In [274]:
df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
df['Years Before Sale'][df['Years Before Sale'] < 0]

2180   -1
Name: Years Before Sale, dtype: int64

In [275]:
df['Years Since Remodel'] = df['Yr Sold'] - df['Year Remod/Add']
df['Years Since Remodel'][df['Years Since Remodel'] < 0]

1702   -1
2180   -2
2181   -1
Name: Years Since Remodel, dtype: int64

We'll drop the negative years which resulted from an input error and remove the original years.

In [276]:
df = df.drop([1702, 2180, 2181], axis=0)
df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)

Next, we'll drop columns that:

* aren't useful for ML
* leak data about the final sale

The columns that leak information about the final sale are those that we wouldn't know at the time we are making the prediction.

In [277]:
## Drop columns that aren't useful
df = df.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

Finally, we'll updated the transform_features() function.

In [278]:
def transform_features(df):
    #Drop columns with more than 5% of missing values
    missing_vals = df.isnull().sum()
    missing_cols = missing_vals[missing_vals>len(df)*0.05].index
    df = df.drop(missing_cols, axis=1)
    
    #Drop text columns that contain missing values
    text_missing_vals = df.select_dtypes(include=['object']).isnull().sum()
    text_missing_cols = text_missing_vals[text_missing_vals>0].index
    df = df.drop(text_missing_cols, axis=1)
    
    #For remaining num columns, fill in missing value with the column mode
    num_cols = df.select_dtypes(include=['int', "float"]).isnull().sum().sort_values(ascending=False)
    num_cols = num_cols[num_cols >= 1]
    mode = df[num_cols.index].mode().to_dict('records')
    df = df.fillna(mode[0])
    
    #Transform years values and drop columns which leak info
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df['Years Since Remodel'] = df['Yr Sold'] - df['Year Remod/Add']
    df = df.drop([1702, 2180, 2181], axis=0)
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
    
    df = df.drop(["PID", "Order"], axis=1)
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return df

Now, let's retrain the model with the transformed features.

In [279]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
final_df = select_features(transform_df)
rmse = train_and_test(final_df)

rmse

55275.36731241307

### Feature selection

We still have 58 features left which is bound to overfit in a linear regression, so we will now do feature selection based on the features correlation and each invidiual column's variance.

Let's generate correlation heatmap matrix of the numerical features in the training data set.

In [280]:
import seaborn as sns
%matplotlib inline

In [281]:
num_df = transform_df.select_dtypes(include=['int', 'float'])

In [282]:
corr = num_df.corr()['SalePrice'].abs().sort_values(ascending=False)
corr

SalePrice              1.000000
Overall Qual           0.801206
Gr Liv Area            0.717596
Garage Cars            0.648361
Total Bsmt SF          0.644012
Garage Area            0.641425
1st Flr SF             0.635185
Years Before Sale      0.558979
Full Bath              0.546118
Years Since Remodel    0.534985
Mas Vnr Area           0.506983
TotRms AbvGrd          0.498574
Fireplaces             0.474831
BsmtFin SF 1           0.439284
Wood Deck SF           0.328183
Open Porch SF          0.316262
Half Bath              0.284871
Bsmt Full Bath         0.276258
2nd Flr SF             0.269601
Lot Area               0.267520
Bsmt Unf SF            0.182751
Bedroom AbvGr          0.143916
Enclosed Porch         0.128685
Kitchen AbvGr          0.119760
Screen Porch           0.112280
Overall Cond           0.101540
MS SubClass            0.085128
Pool Area              0.068438
Low Qual Fin SF        0.037629
Bsmt Half Bath         0.035875
3Ssn Porch             0.032268
Misc Val

We'll keep the numerical columns that have a correlation higher than 0.3

In [283]:
corr[corr>0.3]

SalePrice              1.000000
Overall Qual           0.801206
Gr Liv Area            0.717596
Garage Cars            0.648361
Total Bsmt SF          0.644012
Garage Area            0.641425
1st Flr SF             0.635185
Years Before Sale      0.558979
Full Bath              0.546118
Years Since Remodel    0.534985
Mas Vnr Area           0.506983
TotRms AbvGrd          0.498574
Fireplaces             0.474831
BsmtFin SF 1           0.439284
Wood Deck SF           0.328183
Open Porch SF          0.316262
Name: SalePrice, dtype: float64

In [284]:
transform_df.shape

(2927, 58)

In [285]:
transform_df = transform_df.drop(corr[corr<0.3].index, axis=1)

In [286]:
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [287]:
categorical_features = []
for col in nominal_features:
    if col in transform_df.columns:
        categorical_features.append(col)

In [288]:
categorical_features

['MS Zoning',
 'Street',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Foundation',
 'Heating',
 'Central Air']

In [289]:
unique_values = transform_df[categorical_features].apply(lambda col: len(col.value_counts())).sort_values(ascending=False)
unique_values

Neighborhood    28
Exterior 2nd    17
Exterior 1st    16
Condition 1      9
Roof Matl        8
House Style      8
Condition 2      8
MS Zoning        7
Heating          6
Foundation       6
Roof Style       6
Bldg Type        5
Lot Config       5
Land Contour     4
Central Air      2
Street           2
dtype: int64

When we dummy code column with many hundreds of unique, hundreds of columns will need to be added back to the data frame. Therefore we will only keep those that have less than 5 unique values.

In [290]:
transform_df = transform_df.drop(unique_values[unique_values>5].index, axis=1)

In [291]:
transform_df.shape

(2927, 30)

Next we'll check for low variability in the data by looking at columns that have a few unique values but more than 95% of the values in the column belong to a specific category.

In [292]:
cat_features = []
for col in categorical_features:
    if col in transform_df.columns:
        cat_features.append(col)

In [293]:
cat_features

['Street', 'Land Contour', 'Lot Config', 'Bldg Type', 'Central Air']

In [294]:
for col in cat_features:
    print(transform_df[col].value_counts(normalize=True)*100)
    print("\n")

Pave    99.590024
Grvl     0.409976
Name: Street, dtype: float64


Lvl    89.921421
HLS     4.099761
Bnk     3.928937
Low     2.049880
Name: Land Contour, dtype: float64


Inside     73.044072
Corner     17.423984
CulDSac     6.149641
FR2         2.903997
FR3         0.478305
Name: Lot Config, dtype: float64


1Fam      82.746840
TwnhsE     7.960369
Duplex     3.723949
Twnhs      3.450632
2fmCon     2.118210
Name: Bldg Type, dtype: float64


Y    93.303724
N     6.696276
Name: Central Air, dtype: float64




We will drop the `Street` column.

In [295]:
transform_df.drop('Street', axis=1, inplace=True)

Next, we'll actualy encode the cat features as categorical (now they are `object`).

In [296]:
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')

We can now dummy these features using pandas .get_dummies function.

In [297]:
dummies = pd.get_dummies(transform_df.select_dtypes(include=['category']))

In [298]:
#Concatenate the dataframes
transform_df = pd.concat([transform_df, dummies], axis=1)

#Drop the original cat features
transform_df.drop(text_cols, axis=1, inplace=True)

In [299]:
transform_df.shape

(2927, 72)

Finally, we'll put all the logic so far in the `select_features()` function.

In [300]:
def select_features(df, corr_threshold=0.3, unique_val_threshold=10):
    #Selecting features that have a correlation higher than 0.3 with the target
    num_df = df.select_dtypes(include=['int', 'float'])
    corr = num_df.corr()['SalePrice'].abs().sort_values(ascending=False)
    transform_df = df.drop(corr[corr<corr_threshold].index, axis=1)
    
    #Transforming the text columns
    categorical_features = [
         'MS Zoning',
         'Street',
         'Land Contour',
         'Lot Config',
         'Neighborhood',
         'Condition 1',
         'Condition 2',
         'Bldg Type',
         'House Style',
         'Roof Style',
         'Roof Matl',
         'Exterior 1st',
         'Exterior 2nd',
         'Foundation',
         'Heating',
         'Central Air']
    unique_values = transform_df[categorical_features].apply(lambda col: len(col.value_counts())).sort_values(ascending=False)
    transform_df = transform_df.drop(unique_values[unique_values>unique_val_threshold].index, axis=1)
    
    #Removing low variability features
    cat_features = []
    for col in categorical_features:
        if col in transform_df.columns:
            cat_features.append(col)
    transform_df.drop('Street', axis=1, inplace=True)
    
    #Get dummies for cat variables
    text_cols = transform_df.select_dtypes(include=['object'])
    for col in text_cols:
        transform_df[col] = transform_df[col].astype('category')
    dummies = pd.get_dummies(transform_df.select_dtypes(include=['category']))
    
    #Concatenate the dataframes
    transform_df = pd.concat([transform_df, dummies], axis=1)

    #Drop the original cat features
    transform_df.drop(text_cols, axis=1, inplace=True)
    
    return transform_df

In [301]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
final_df = select_features(transform_df)
rmse = train_and_test(final_df)

rmse

33278.76501848715

Let's decrease the threshold for unique variables in the categorical columns to 5.

In [302]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
final_df = select_features(transform_df, 0.3, 5 )
rmse = train_and_test(final_df)

rmse

32904.16085766734

### Cross validation

For the final part, we will perform k-fold cross validation.

We will update the `train_and_test` function to take an optional input `k` for number of folds.

In [342]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    model = LinearRegression()
    
    #If k=0, implement a holdout validation
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        model.fit(train[features], train["SalePrice"])
        pred = model.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], pred)
        rmse = np.sqrt(mse)

        return rmse
    
    if k > 0:
        folds = KFold(n_splits=k, shuffle=True)
        rmse_vals = []
        i=0
        for train_ix, test_ix, in folds.split(df):
            i += 1
            train = df.iloc[train_ix]
            test = df.iloc[test_ix]
            model.fit(train[features], train["SalePrice"])
            pred = model.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], pred)
            rmse = np.sqrt(mse)
            rmse_vals.append(rmse)
            print("Fold {}: {}".format(i, rmse))
        average_rmse = np.mean(rmse_vals)
        print("Average: {} ".format(average_rmse))
        
        return average_rmse

In [345]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
final_df = select_features(transform_df, 0.3, 5 )
rmse = train_and_test(final_df, k=10)

Fold 1: 27632.699089861715
Fold 2: 23344.792308420587
Fold 3: 29164.435907126324
Fold 4: 23729.5439951428
Fold 5: 24035.673865097604
Fold 6: 24277.88329240221
Fold 7: 30298.739255807814
Fold 8: 47821.79058172581
Fold 9: 22799.754528317902
Fold 10: 33244.171625548406
Average: 28634.948444945116 


We observe that 10 k-fold cross validation performs better than a simple holdout validation: RMSE - 28634 vs RMSE - 32904. The most important skill when it comes to feature engineering is domain expertise which helps in coming up with new ways of extracting predictive information from the orignal variables.